# 🎬 Analyse IMDb - Préparation des données

**Objectif** : Créer un dataset propre avec films, notes, genres et acteurs/réalisateurs

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import os

# Configuration
pd.set_option('display.max_columns', None)

## 📁 Configuration des chemins (relatifs pour GitHub)

In [ ]:
# Chemins relatifs (compatible GitHub)
PROJECT_ROOT = Path.cwd().parent if 'notebooks' in str(Path.cwd()) else Path.cwd()
DATA_DIR = PROJECT_ROOT / "data"
PARQUET_DIR = DATA_DIR / "PARQUETS"

# Créer les dossiers
PARQUET_DIR.mkdir(parents=True, exist_ok=True)
print(f"✅ Dossier créé : {PARQUET_DIR}")

## 📥 ÉTAPE 1 : Charger et filtrer title.basics

In [ ]:
# URL IMDb
url_basics = "https://datasets.imdbws.com/title.basics.tsv.gz"

# Chemin de sauvegarde
parquet_basics = PARQUET_DIR / "title_basics_movies.parquet"

# Paramètres
chunk_size = 500_000
colonnes_utiles = [
    'tconst', 'titleType', 'primaryTitle', 'originalTitle',
    'isAdult', 'startYear', 'runtimeMinutes', 'genres'
]

# Lecture par chunks
chunks_filtres = []
print("🔄 Traitement de title.basics...")

for i, chunk in enumerate(pd.read_csv(
    url_basics,
    sep='\t',
    chunksize=chunk_size,
    usecols=colonnes_utiles,
    dtype={'startYear': str, 'runtimeMinutes': str},
    na_values='\\N'
), 1):
    
    # ✅ FILTRES COMPLETS
    chunk_filtre = chunk[
        (chunk['titleType'] == 'movie') &
        (chunk['isAdult'] == '0') &  # ← IMPORTANT : Exclure films adultes
        (chunk['startYear'].notna())
    ].copy()
    
    # Conversions
    chunk_filtre['startYear'] = pd.to_numeric(chunk_filtre['startYear'], errors='coerce')
    chunk_filtre['runtimeMinutes'] = pd.to_numeric(chunk_filtre['runtimeMinutes'], errors='coerce')
    
    # Filtre sur les années (après 1970)
    chunk_filtre = chunk_filtre[chunk_filtre['startYear'] > 1970]
    
    chunks_filtres.append(chunk_filtre)
    print(f"  Chunk {i} : {len(chunk_filtre):,} films gardés")

# Concaténation
df_movies = pd.concat(chunks_filtres, ignore_index=True)
print(f"\n✅ Total : {len(df_movies):,} films conservés")
print(f"📊 Période : {df_movies['startYear'].min():.0f} - {df_movies['startYear'].max():.0f}")

# Sauvegarde
df_movies.to_parquet(parquet_basics, engine='pyarrow', compression='snappy', index=False)
print(f"💾 Sauvegardé : {parquet_basics}")

## 📥 ÉTAPE 2 : Charger title.ratings

In [ ]:
# URL IMDb
url_ratings = "https://datasets.imdbws.com/title.ratings.tsv.gz"
parquet_ratings = PARQUET_DIR / "title_ratings.parquet"

print("🔄 Traitement de title.ratings...")

# Lecture
df_ratings = pd.read_csv(url_ratings, sep='\t', na_values='\\N')

# Filtrer pour garder seulement nos films
tconst_movies = set(df_movies['tconst'])
df_ratings = df_ratings[df_ratings['tconst'].isin(tconst_movies)]

print(f"✅ {len(df_ratings):,} ratings conservés")

# Sauvegarde
df_ratings.to_parquet(parquet_ratings, engine='pyarrow', compression='snappy', index=False)
print(f"💾 Sauvegardé : {parquet_ratings}")

## 🔗 ÉTAPE 3 : Créer le dataset complet avec genres encodés

In [ ]:
# Jointure movies + ratings
print("🔗 Jointure movies + ratings...")
df_complet = df_movies.merge(df_ratings, on='tconst', how='left')

print(f"✅ Dataset complet : {len(df_complet):,} films")
print(f"📊 Films avec notes : {df_complet['averageRating'].notna().sum():,}")

# Encoder les genres
print("\n🎭 Encodage des genres...")
df_complet['genres'] = df_complet['genres'].fillna('')
df_complet = pd.concat([
    df_complet,
    df_complet['genres'].str.get_dummies(sep=',')
], axis=1)

genre_cols = [col for col in df_complet.columns if col not in df_movies.columns and col not in df_ratings.columns]
print(f"✅ {len(genre_cols)} genres encodés")

# Aperçu
print("\n🔍 Aperçu :")
display(df_complet.head())

# Sauvegarde intermédiaire
parquet_complet = PARQUET_DIR / "imdb_complet_base.parquet"
df_complet.to_parquet(parquet_complet, engine='pyarrow', compression='snappy', index=False)
print(f"💾 Sauvegardé : {parquet_complet}")

## 🎭 ÉTAPE 4 : Ajouter les acteurs et réalisateurs

In [ ]:
# URL IMDb
url_principals = "https://datasets.imdbws.com/title.principals.tsv.gz"

# Paramètres
chunks_filtres = []
chunk_size = 500_000

print("🔄 Traitement de title.principals (peut prendre 10 min)...")

for i, chunk in enumerate(pd.read_csv(
    url_principals,
    sep='\t',
    chunksize=chunk_size,
    usecols=['tconst', 'ordering', 'nconst', 'category'],
    na_values='\\N'
), 1):
    
    # Filtre : Top 5 acteurs + tous les réalisateurs
    chunk_filtre = chunk[
        (chunk['tconst'].isin(tconst_movies)) &
        (
            ((chunk['category'].isin(['actor', 'actress'])) & (chunk['ordering'] <= 5)) |
            (chunk['category'] == 'director')
        )
    ].copy()
    
    chunks_filtres.append(chunk_filtre)
    
    if i % 10 == 0:
        print(f"  {i} chunks traités")

df_principals = pd.concat(chunks_filtres, ignore_index=True)
print(f"\n✅ {len(df_principals):,} lignes conservées")

# Sauvegarder
parquet_principals = PARQUET_DIR / "title_principals_filtered.parquet"
df_principals.to_parquet(parquet_principals, engine='pyarrow', compression='snappy', index=False)
print(f"💾 Sauvegardé : {parquet_principals}")

In [ ]:
# Charger les noms
url_names = "https://datasets.imdbws.com/name.basics.tsv.gz"
nconst_needed = set(df_principals['nconst'])

print("🔄 Chargement des noms...")
chunks_names = []

for i, chunk in enumerate(pd.read_csv(
    url_names,
    sep='\t',
    chunksize=500_000,
    usecols=['nconst', 'primaryName'],
    na_values='\\N'
), 1):
    chunks_names.append(chunk[chunk['nconst'].isin(nconst_needed)])
    if i % 10 == 0:
        print(f"  {i} chunks traités")

df_names = pd.concat(chunks_names, ignore_index=True)
print(f"✅ {len(df_names):,} noms récupérés")

# Sauvegarder
parquet_names = PARQUET_DIR / "name_basics_filtered.parquet"
df_names.to_parquet(parquet_names, engine='pyarrow', compression='snappy', index=False)
print(f"💾 Sauvegardé : {parquet_names}")

In [ ]:
# Joindre les noms
print("🔗 Jointure principals + names...")
df_principals_noms = df_principals.merge(df_names, on='nconst', how='left')

# Créer les listes d'acteurs et réalisateurs par film
acteurs = df_principals_noms[
    df_principals_noms['category'].isin(['actor', 'actress'])
].groupby('tconst')['primaryName'].apply(list).reset_index(name='acteurs')

realisateurs = df_principals_noms[
    df_principals_noms['category'] == 'director'
].groupby('tconst')['primaryName'].apply(list).reset_index(name='realisateurs')

print(f"✅ {len(acteurs):,} films avec acteurs")
print(f"✅ {len(realisateurs):,} films avec réalisateurs")

# Joindre à df_complet
df_complet = df_complet.merge(acteurs, on='tconst', how='left')
df_complet = df_complet.merge(realisateurs, on='tconst', how='left')

# Remplacer NaN par listes vides
df_complet['acteurs'] = df_complet['acteurs'].apply(lambda x: x if isinstance(x, list) else [])
df_complet['realisateurs'] = df_complet['realisateurs'].apply(lambda x: x if isinstance(x, list) else [])

print(f"\n✅ Dataset final : {df_complet.shape}")

## 💾 ÉTAPE 5 : Sauvegarde finale

In [ ]:
# Sauvegarde finale
parquet_final = PARQUET_DIR / "imdb_complet_avec_cast.parquet"
df_complet.to_parquet(parquet_final, engine='pyarrow', compression='snappy', index=False)

print("="*60)
print("✅ TRAITEMENT TERMINÉ !")
print("="*60)
print(f"\n💾 Dataset final : {parquet_final}")
print(f"📊 Dimensions : {df_complet.shape}")
print(f"\n📋 Statistiques :")
print(f"  • Films : {len(df_complet):,}")
print(f"  • Avec notes : {df_complet['averageRating'].notna().sum():,}")
print(f"  • Avec acteurs : {df_complet['acteurs'].apply(len).gt(0).sum():,}")
print(f"  • Avec réalisateurs : {df_complet['realisateurs'].apply(len).gt(0).sum():,}")

print("\n🔍 Aperçu final :")
display(df_complet[['primaryTitle', 'startYear', 'genres', 'averageRating', 'acteurs', 'realisateurs']].head())